data collected April 25, 2020 on a DigitalOcean droplet running Ubuntu 18.04

In [1]:
# the order of installation matters for these two :(
!pip install python-geoip-geolite2
!pip install python-geoip-python3

In [2]:
import pandas
print('pandas',pandas.__version__)
import re
import time
from geoip import geolite2 # https://pythonhosted.org/python-geoip/
import sys
print(sys.version)
import matplotlib.pyplot as plt




pandas 0.23.4
3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [3]:
# https://stackoverflow.com/questions/30700166/python-open-file-error
with open('nginx_error.log','r', errors = 'ignore') as f:
    lines = f.readlines()

In [4]:
lines[0:10]

['2020/04/25 21:26:56 [info] 6#6: *29 client 162.129.252.228 closed keepalive connection\n',
 '2020/04/25 21:26:56 [info] 6#6: *25 client 162.129.252.228 closed keepalive connection\n',
 '2020/04/25 21:26:56 [info] 6#6: *26 client 162.129.252.228 closed keepalive connection\n',
 '2020/04/25 21:26:56 [info] 6#6: *22 client 162.129.252.228 closed keepalive connection\n',
 '2020/04/25 21:36:05 [info] 6#6: *22 client closed connection while waiting for request, client: 96.245.195.226, server: 0.0.0.0:80\n',
 '2020/04/25 21:36:36 [info] 6#6: *45 client closed connection while waiting for request, client: 96.245.195.226, server: 0.0.0.0:80\n',
 '2020/04/25 21:36:41 [info] 6#6: *61 client closed connection while waiting for request, client: 96.245.195.226, server: 0.0.0.0:80\n',
 '2020/04/25 21:36:48 [info] 6#6: *65 client closed connection while waiting for request, client: 96.245.195.226, server: 0.0.0.0:80\n',
 '2020/04/25 21:37:01 [info] 6#6: *67 client closed connection while waiting for

In [5]:
list_of_dicts = []
for indx, line in enumerate(lines):
    line_as_dict = {}
    line_as_list = line.split(' ')
    line_as_dict['date time'] = ' '.join(line_as_list[0:2])
    line_as_dict['msg type'] = line_as_list[2].replace('[','').replace(']','')
    # https://stackoverflow.com/a/26125951/1164295
    msg = line_as_list[3:]
    line_as_dict['proc id'] = msg[0].split('#')[0]
    line_as_dict['thread id'] = msg[0].split('#')[1].replace(':','')
    if msg[1].startswith('*'):
        line_as_dict['cid'] = msg[1].replace('*','')
        msg_text = ' '.join(msg[2:]).strip()
    else:
        msg_text = ' '.join(msg[1:]).strip()
    #print(msg_text.split(', '))
    unparsed_text = ''
    for this_text in msg_text.split(', '):
        if this_text.startswith('request: '):
            line_as_dict['request'] = this_text.replace('request: ','')
        elif this_text.startswith('server: '):
            line_as_dict['server'] = this_text.replace('server: ','')
        elif this_text.startswith('client: '):
            line_as_dict['client'] = this_text.replace('client: ','')
        else:
            unparsed_text += this_text
    line_as_dict['unparsed text'] = unparsed_text
    
    list_of_dicts.append(line_as_dict)

In [6]:
df = pandas.DataFrame(list_of_dicts)
df.shape

(412, 9)

In [7]:
# https://strftime.org/
df['datetime'] = pandas.to_datetime(df['date time'], format='%Y/%m/%d %H:%M:%S')
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
df.drop('date time', inplace=True, axis=1)

In [8]:
df.head()

,cid,client,msg type,proc id,request,server,thread id,unparsed text,datetime
0,29,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56
1,25,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56
2,26,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56
3,22,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56
4,22,96.245.195.226,info,6,NaN,0.0.0.0:80,6,client closed connection while waiting for req...,2020-04-25 21:36:05


In [9]:
df['request'].value_counts()

"GET / HTTP/1.1"                                      4
"H   tj#D+l׍J
nxu[lE
-jxLru%Rtgfv]%̀
Ϯ fȍD\t"    1
"POST /cgi-bin/mainfunction.cgi HTTP/1.1"             1
"CONNECT ip.ws.126.net:443 HTTP/1.1"                  1
"sh+/tmp/gpon80&ipv=0"                                1
"/tmp/gpon80&ipv=0"                                   1
Name: request, dtype: int64

In [10]:
df['server'].value_counts()

0.0.0.0:80    32
               9
Name: server, dtype: int64

In [11]:
df['client'].value_counts()

96.245.195.226    24
35.197.133.35      3
85.51.57.203       2
47.101.56.34       2
223.149.7.214      1
118.173.190.27     1
106.52.178.254     1
148.63.179.102     1
45.13.93.82        1
42.231.98.143      1
113.162.186.4      1
1.1.162.136        1
70.76.233.21       1
66.240.205.34      1
Name: client, dtype: int64

In [33]:
# download country code lookup table from https://dev.maxmind.com/geoip/legacy/codes/iso3166/
# https://dev.maxmind.com/static/csv/codes/iso3166.csv
    
country_code_df = pandas.read_csv('iso3166.csv', header=None, names=['country code', 'name'])
#country_code_df.shape    
    
def country_if_ip(ip):
    ip = str(ip)
    if ip!='nan':
        try:
            match = geolite2.lookup(ip)
        except ValueError as err:
            print((ip))
        if match:
            return country_code_df[country_code_df['country code']==match.country]['name'].values[0]
        else:
            return None
    else:
        return None

In [34]:
df['client country'] = df['client'].apply(country_if_ip)

In [35]:
df.head()

,cid,client,msg type,proc id,request,server,thread id,unparsed text,datetime,client country
0,29,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56,None
1,25,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56,None
2,26,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56,None
3,22,NaN,info,6,NaN,NaN,6,client 162.129.252.228 closed keepalive connec...,2020-04-25 21:26:56,None
4,22,96.245.195.226,info,6,NaN,0.0.0.0:80,6,client closed connection while waiting for req...,2020-04-25 21:36:05,United States


In [36]:
df['client country'].value_counts()

United States    29
China             3
Canada            3
Spain             2
Thailand          2
Vietnam           1
Name: client country, dtype: int64